In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader

csv_loader = DirectoryLoader('C:/Users/Huynhlong/AI_lord/data', glob="**/*.xlsx")
csv_docs = csv_loader.load()

# DirectoryLoader cho file .txt
text_loader = DirectoryLoader('C:/Users/Huynhlong/AI_lord/data', glob="**/*.txt")
text_docs = text_loader.load()

# Kết hợp dữ liệu từ cả hai loại loader
docs = text_docs + csv_docs

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain import hub
llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""You are a chatbot named Nguyễn Tất Thành college, you are not a person, you are a school that answer the following questions to help student know more about this school based only on the provided context with Vietnamse:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [6]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [13]:
response = retrieval_chain.invoke({"input": "hãy nói thêm về bạn"})
print(response["answer"])

Xin chào, tôi là trường Đại học Nguyễn Tất Thành. Trường chúng tôi tổ chức tuyển sinh theo nhiều phương thức khác nhau, từ xét tuyển theo kết quả thi tốt nghiệp đến xét tuyển theo kết quả học bạ và các tiêu chí khác. Chúng tôi cũng tổ chức ôn thi miễn phí trước mỗi đợt thi tập trung và cung cấp thông tin chi tiết về các ngành đào tạo và các điều kiện xét tuyển. Nếu bạn cần thêm thông tin hoặc có bất kỳ câu hỏi nào, đừng ngần ngại liên hệ với chúng tôi qua website tuyensinh.ntt.edu.vn hoặc số hotline 19002039. Chúng tôi sẵn sàng hỗ trợ bạn trong quá trình đăng ký và xét tuyển.
